# 🔌 bitcoin.API.ipynb
This notebook demonstrates how to:
- Fetch live Bitcoin data from the [CoinGecko API](https://www.coingecko.com/en/api)
- Format it to match the Protobuf schema (`bitcoin_full.proto`)
- Serialize it using `bitcoin_full_pb2.py`
- Save to disk and read it back

In [1]:
import sys, os
sys.path.append(os.path.join(os.getcwd(), 'src'))

import requests
import time
from bitcoin_full_pb2 import BitcoinFullData

## 📡 Step 1: Fetch Bitcoin Data from CoinGecko

In [2]:
url = "https://api.coingecko.com/api/v3/coins/markets"
params = {"vs_currency": "usd", "ids": "bitcoin"}
response = requests.get(url, params=params)
btc = response.json()[0]  # Extract first (and only) item
btc

{'id': 'bitcoin',
 'symbol': 'btc',
 'name': 'Bitcoin',
 'image': 'https://coin-images.coingecko.com/coins/images/1/large/bitcoin.png?1696501400',
 'current_price': 103740,
 'market_cap': 2060807468936,
 'market_cap_rank': 1,
 'fully_diluted_valuation': 2060807468936,
 'total_volume': 49978884695,
 'high_24h': 103890,
 'low_24h': 99239,
 'price_change_24h': 4317.98,
 'price_change_percentage_24h': 4.3431,
 'market_cap_change_24h': 88124332562,
 'market_cap_change_percentage_24h': 4.46723,
 'circulating_supply': 19862190.0,
 'total_supply': 19862190.0,
 'max_supply': 21000000.0,
 'ath': 108786,
 'ath_change_percentage': -4.62573,
 'ath_date': '2025-01-20T09:11:54.494Z',
 'atl': 67.81,
 'atl_change_percentage': 152908.30405,
 'atl_date': '2013-07-06T00:00:00.000Z',
 'roi': None,
 'last_updated': '2025-05-09T08:06:18.151Z'}

## 🛠 Step 2: Format Data for Protobuf

In [3]:
btc_dict = {
    "timestamp": int(time.time()),
    "id": btc["id"],
    "symbol": btc["symbol"],
    "name": btc["name"],
    "image": btc["image"],
    "current_price": btc["current_price"],
    "market_cap": btc["market_cap"],
    "market_cap_rank": btc["market_cap_rank"],
    "fully_diluted_valuation": btc.get("fully_diluted_valuation", 0.0),
    "total_volume": btc["total_volume"],
    "high_24h": btc["high_24h"],
    "low_24h": btc["low_24h"],
    "price_change_24h": btc["price_change_24h"],
    "price_change_percentage_24h": btc["price_change_percentage_24h"],
    "market_cap_change_24h": btc["market_cap_change_24h"],
    "market_cap_change_percentage_24h": btc["market_cap_change_percentage_24h"],
    "circulating_supply": btc["circulating_supply"],
    "total_supply": btc.get("total_supply", 0.0),
    "max_supply": btc.get("max_supply", 0.0),
    "ath": btc["ath"],
    "ath_change_percentage": btc["ath_change_percentage"],
    "ath_date": btc["ath_date"],
    "atl": btc["atl"],
    "atl_change_percentage": btc["atl_change_percentage"],
    "atl_date": btc["atl_date"],
    "last_updated": btc["last_updated"],
    "source": "CoinGecko"
}

btc_dict

{'timestamp': 1746778048,
 'id': 'bitcoin',
 'symbol': 'btc',
 'name': 'Bitcoin',
 'image': 'https://coin-images.coingecko.com/coins/images/1/large/bitcoin.png?1696501400',
 'current_price': 103740,
 'market_cap': 2060807468936,
 'market_cap_rank': 1,
 'fully_diluted_valuation': 2060807468936,
 'total_volume': 49978884695,
 'high_24h': 103890,
 'low_24h': 99239,
 'price_change_24h': 4317.98,
 'price_change_percentage_24h': 4.3431,
 'market_cap_change_24h': 88124332562,
 'market_cap_change_percentage_24h': 4.46723,
 'circulating_supply': 19862190.0,
 'total_supply': 19862190.0,
 'max_supply': 21000000.0,
 'ath': 108786,
 'ath_change_percentage': -4.62573,
 'ath_date': '2025-01-20T09:11:54.494Z',
 'atl': 67.81,
 'atl_change_percentage': 152908.30405,
 'atl_date': '2013-07-06T00:00:00.000Z',
 'last_updated': '2025-05-09T08:06:18.151Z',
 'source': 'CoinGecko'}

## 📦 Step 3: Serialize to Protobuf Format

In [4]:
msg = BitcoinFullData(**btc_dict)
serialized = msg.SerializeToString()
print(f"✅ Serialized {len(serialized)} bytes")

✅ Serialized 364 bytes


## 💾 Step 4: Save to Disk and Read Back

In [5]:
file_path = "src/data/sample_single_record.pb"
with open(file_path, "wb") as f:
    f.write(len(serialized).to_bytes(4, 'little'))
    f.write(serialized)

# Read back
with open(file_path, "rb") as f:
    length_bytes = f.read(4)
    size = int.from_bytes(length_bytes, 'little')
    message_data = f.read(size)

msg_read = BitcoinFullData()
msg_read.ParseFromString(message_data)
msg_read

timestamp: 1746778048
id: "bitcoin"
symbol: "btc"
name: "Bitcoin"
image: "https://coin-images.coingecko.com/coins/images/1/large/bitcoin.png?1696501400"
current_price: 103740.0
market_cap: 2060807468936.0
market_cap_rank: 1
fully_diluted_valuation: 2060807468936.0
total_volume: 49978884695.0
high_24h: 103890.0
low_24h: 99239.0
price_change_24h: 4317.98
price_change_percentage_24h: 4.3431
market_cap_change_24h: 88124332562.0
market_cap_change_percentage_24h: 4.46723
circulating_supply: 19862190.0
total_supply: 19862190.0
max_supply: 21000000.0
ath: 108786.0
ath_change_percentage: -4.62573
ath_date: "2025-01-20T09:11:54.494Z"
atl: 67.81
atl_change_percentage: 152908.30405
atl_date: "2013-07-06T00:00:00.000Z"
last_updated: "2025-05-09T08:06:18.151Z"
source: "CoinGecko"